# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load CSV
weather_data = pd.read_csv(
    "weather_data.csv", dtype="object", encoding="utf-8")

#Check it loaded
weather_data.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind speed
0,0,saldanha,ZA,-33.01,17.94,59.0,62,0,17.22
1,1,luderitz,NaN,-26.65,15.16,62.15,51,100,29.68
2,2,yellowknife,CA,62.46,-114.35,19.4,85,75,11.41
3,3,qaanaaq,GL,77.48,-69.36,14.9,83,99,5.95
4,4,vaini,TO,-21.2,-175.2,71.6,68,75,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#Create locations
weather_data["Humidity"] = pd.to_numeric(weather_data["Humidity"])
weather_data["Temperature"] = pd.to_numeric(weather_data["Temperature"])
weather_data["Wind speed"] = pd.to_numeric(weather_data["Wind speed"])
weather_data["Wind speed"] = pd.to_numeric(weather_data["Cloudiness"])
locations = weather_data[["Latitude", "Longitude"]].astype(float)

# Create a Heatmap layer
vac_fig = gmaps.figure()
humidity = weather_data["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

vac_fig.add_layer(heat_layer)

vac_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_data.dtypes

Unnamed: 0      object
City            object
Country         object
Latitude        object
Longitude       object
Temperature    float64
Humidity         int64
Cloudiness      object
Wind speed       int64
dtype: object

In [6]:
#Create DF w/ ideal weather criteria by dropping those outside of parameters
vaca_data = weather_data.loc[[weather_data[(weather_data['Temperature']>75)] & (weather_data['Temperature']<90) & (weather_data['Wind speed']<10) & (weather_data['Cloudiness']==0)].index, inplace = False]
#vaca_data = weather_data.drop(weather_data[(weather_data['Temperature'] < 75) & (weather_data['Temperature'] > 90) & (weather_data['Wind speed'] > 10) & (weather_data['Cloudiness'] > 0)].index, inplace = False)
vaca_data_clean = vaca_data.dropna()
vaca_data_clean.head()

SyntaxError: invalid syntax (<ipython-input-6-5897d99b4d3c>, line 2)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#Store variable
vaca_data_clean['Hotel name'] = ""

#base url and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "lodging",
    "radius": 5000,
    "keyword":"hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in vaca_data_clean.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        vaca_data_clean.loc[index, 'Hotel name'] = results[0]['name']
        vaca_data_clean.loc[index, 'Hotel latitude'] = results['geometry']['location']['lat']
        vaca_data_clean.loc[index, 'Hotel longitude']=results['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vaca_data_clean.iterrows()]
locations = vaca_data_clean[["Hotel latitude", "Hotel longitude"]]

In [ ]:
# Create a map using coordinates to set markers
marker_locations = vaca_data_clean[['Latitude', 'Longitude']]

# Create a marker_layer 
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)

In [ ]:
# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig